In [1]:
!pip install --upgrade plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.2 MB 9.0 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


In [2]:
#필요한 모듈 및 라이브러리 불러오기
import plotly.express as px

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#검색 데이터 및 DB 데이터 로드 
# 호텔의 경우 활성화 되어 있는 숙소만 사용
search = pd.read_excel('/content/drive/MyDrive/cp2/검색페이지EDA_by_JYP.xlsx')
ht = pd.read_excel('/content/drive/MyDrive/cp2/allmytour_DB_data.xlsx')
product = pd.read_excel('/content/drive/MyDrive/cp2/allmytour_DB_data.xlsx', sheet_name = -1)
ht = ht.loc[ht.avail == 1, :]
ht.rename(columns={'idx':'product_idx',
                   'address_1_idx':'city'}, inplace= True) # table 병합을 위해 이름 재설정,table 이름 재설정 address_1_1dx => 광역 단위 지역을 나타냄  
table = pd.merge(product, ht, how = 'left', on = 'product_idx') #호텔의 목록과 유저의 구매기록을 병합 
condition1 = (table.order_product_status == 'confirm') # 테이블의 경우 결제완료된 결제들만 활용
table = table.loc[condition1, :]

# 더 직관적으로 보일 수 있게 지역코드를 지역 명으로 변경 
def city(arg):
    if arg == 11:
        arg = '서울'
    elif arg == 21:
        arg = '부산'
    elif arg == 22:
        arg = '대구'
    elif arg == 23:
        arg = '인천'
    elif arg == 24:
        arg = '광주'
    elif arg == 25:
        arg = '대전'
    elif arg == 26:
        arg = '울산'
    elif arg == 29:
        arg = '세종'
    elif arg == 31:
        arg = '경기'
    elif arg == 32:
        arg = '강원'
    elif arg == 33:
        arg = '충북'
    elif arg == 34:
        arg = '충남'
    elif arg == 35:
        arg = '전북'
    elif arg == 36:
        arg = '전남'
    elif arg == 37:
        arg = '경북'
    elif arg == 38:
        arg = '경남'
    elif arg == 39:
        arg = '제주'
    return arg 

table.city = table.city.apply(city)
ht.city = ht.city.apply(city)

#데이터 프레임 확인 
table.head()



/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,idx,order_num,order_product_status,user_idx,checkin_date,checkout_date,terms,adt_cnt,chd_cnt,room_cnt,...,nation_idx,city,address_2_idx,checkin,checkout,max_child_age,max_free_child_age,free_child_num,name_en,tourNumber
8,18184,22042000437,confirm,11,2022-05-22,2022-05-23,1,2,0,1,...,KR,서울,11190.0,15:00:00,12:00:00,6.0,0.0,0.0,NaN,0.0
9,32656,22060800255,confirm,11,2022-07-09,2022-07-10,1,2,0,1,...,KR,강원,32350.0,15:00:00,11:00:00,6.0,6.0,1.0,NaN,11610.0
12,23124,22042800041,confirm,23,2022-05-03,2022-05-04,1,2,0,1,...,KR,서울,11140.0,15:00:00,11:00:00,6.0,6.0,1.0,NaN,11383.0
16,7724,22041200133,confirm,26,2022-04-30,2022-05-01,1,2,0,1,...,KR,서울,11190.0,15:00:00,11:00:00,6.0,6.0,1.0,NaN,11848.0
17,7727,22041200136,confirm,26,2022-05-19,2022-05-20,1,2,0,1,...,KR,서울,11030.0,15:00:00,11:00:00,6.0,6.0,1.0,NaN,11387.0


검색 데이터를 바탕으로 한 고객 수요 파악
- 검색 데이터를 올마이투어의 기존 검색기능을 바탕으로 유저들이 어떤 지역의 숙소를 가장 많이 찾아봤는지 확인이 가능 
- 아래와 같은 결과는 분석팀에서 분석한 결과를 바탕으로 미루어 보았을 때, 서울 경기 지역의 유저가 가장 많기 때문에 이와 같다고 볼 수 있음 
    - 서울지역이 36248회로 8376회의 강원지역 숙소에 비해 4배 더 많이 검색됨
    - 전체 검색량은 71414회 

In [4]:
#Plotly를 사용하기 위한 라이브러리 모듈 로드 
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.offline as pyo

from plotly.subplots import make_subplots


In [5]:
#유저들이 가장 많이 검색한 지역 
search.paramLocationText.value_counts().head(10)
search_top_city = search.paramLocationText.value_counts()[0:5]

#시각화 
fig = px.bar(search_top_city,title="각 지역 별 검색 횟수")
fig.update_layout(
    title_text='각 지역 별 검색 횟수', # title of plot\
    xaxis_title_text='숙소 지역', # xaxis label\
    yaxis_title_text='검색 횟수' # yaxis label\,
)
fig.show()

In [6]:
#데이터 베이스에 저장되어있는 지역 별 숙소의 수와 실제 구매로 이어지는 지역 별 숙소의 수
top_5_DB = ht.city.value_counts()[0:5]
top_5_confirmed = table.city.value_counts()[0:5]# 가장 많이 팔린 지역 5개

In [7]:
top_5_DB

강원    462
제주    359
경기    293
서울    243
부산    215
Name: city, dtype: int64

In [8]:
#데이터 베이스에 저장되어있는 지역 별 숙소의 수와 실제 구매로 이어지는 지역 별 숙소의 수
top_6_DB = ht.city.value_counts()[0:6]
top_6_confirmed = table.city.value_counts()[0:6]# 가장 많이 팔린 지역 5개

In [9]:
top_6_DB = pd.DataFrame(top_6_DB)
top_6_confirmed = pd.DataFrame(top_6_confirmed)
top_6_DB

,city
강원,462
제주,359
경기,293
서울,243
부산,215
경북,129


In [10]:
#데이터 베이스에 저장되어있는 지역 별 숙소의 수와 실제 구매로 이어지는 지역 별 숙소의 수
trace3 = go.Bar(x=top_6_DB.index, y=top_6_DB.city.values, name='DB 등록 숙박 등급  수량 분석')
trace4 = go.Bar(x=top_6_confirmed.index, y=top_6_confirmed.values, name='구매 완료 숙박 등급 수량 분석')

fig1 = px.bar(top_6_DB,title="DB에 저장된 지역 별 숙박의 수량")
fig1.update_layout(
    title_text='DB 저장 지역별 숙박', # title of plot\
    xaxis_title_text='숙소 지역', # xaxis label\
    yaxis_title_text='보유 수량' # yaxis label\,
)
fig1.show()



In [11]:
fig7 = px.bar(top_6_confirmed,title=" 지역 별 숙박의 판매 수량")
fig7.update_layout(
    title_text='지역 별 숙박의 판매 수량', # title of plot\
    xaxis_title_text='숙소 지역', # xaxis label\
    yaxis_title_text='판매 수량' # yaxis label\,
)
fig7.show()

위의 결과를 바탕으로 판매 수량이 1000을 초과하는 서울 강원 지역과, 1000 미만인 기타 지역을 나눠서 문제상황을 분석한다.
- 1000초과 지역: 서울을 중심으로
    - 서울의 경우 판매량이 검색량의 1/10도 되지 않는다. 강원도는 판매량이 검색량의 2/10 수준인데, 수요에 비해 공급이 적기 때문으로 풀이된다.  
- 1000미만 지역: 제주, 대구,경북, 기타지역을 중심으로  
    - 제주도의 경우 DB에 300개가 넘는 숙박이 활성화 되어 있는데, 실제 구매로 이어진 숫자는 26건이다.
        - 이는 두가지 분석으로 이어질 수 있다. 
        - 첫 번째는 등록되어 있는 숙박들이 유저의 니즈를 충족시키지 못하는 것
        - 두 번째는 수많은 숙박 정보 중 유저들이 원하는 숙박정보를 효과적으로 제공하지 못하는 것을 의미한다. 
    - 대구 경북지역은 이탈률이 가장 높은 지역이라는 분석팀의 분석이 있었다. 
    - 기타 판매량 상위6개 도시중 서울 경기를 제외하면 모두 해안가에 인접해 있는 지역이다. 
        - 최근 해안가에는 

서울 문제점
- 가상의 개념 PPS(Purchase Per Search)가 있다고 하자. 이 개념은 지역 별 검색 수량 대비 구매로 이어진 횟수다.
- PPS = 구매수량 / 검색 횟수 
- PPS가 낮다는 것이 시사하는 바는 다음과 같다. 
    - 유저가 원하는 지역의 상품을 사기까지에 많은 검색이 필요하다 -> UX 감소
    - 해당 지역의 수요에 비해 니즈를 충족 시키는 적절한 공급이 없다.
    - 서울은 PPS가 검색량 2위인 강원 지역 PPS값의 1/2 수준이다.   

In [12]:
#필요한 자료 가져오기 
search = pd.DataFrame(search_top_city)
order = pd.DataFrame(top_5_confirmed)
search.rename(columns={'paramLocationText':'search'}, inplace= True)
order.rename(columns={'city':'purchase'}, inplace= True)
PPS = search.join(order)

PPS.dropna( inplace= True)


In [13]:
#PPS 값 설정
PPS['PPS'] = PPS.purchase/PPS.search
PPS_need = PPS.iloc[0:2, -1]
PPS_need
# 시각화 
fig2 = px.bar(PPS_need)
fig2.update_layout(
    title_text='PPS', # title of plot\
    xaxis_title_text='지역', # xaxis label\
    yaxis_title_text='PPS' # yaxis label\,
)
fig2.show()

In [14]:
table1 = pd.merge(product, ht, how = 'left', on = 'product_idx') #호텔의 목록과 유저의 구매기록을 병합 
table1[table1.city == '제주'].order_product_status.value_counts()

cancel     44
confirm    26
fail        6
Name: order_product_status, dtype: int64

제주지역의 문제점

In [15]:
#필요한 데이터 가져오기 
#제주 vs 인천비교
top_10_DB = ht.city.value_counts()[0:10]
top_10_confirmed = table.city.value_counts()[0:10]
order_10 = pd.DataFrame(top_10_confirmed)
DB_10 = pd.DataFrame(top_10_DB)
DB_10.rename(columns={'city':'DB'}, inplace= True)
order_10.rename(columns={'city':'purchase'}, inplace= True)
jjic = DB_10.join(order_10) # 제주와 인천을 비교하기 위한 테이블 만들기 
jjic = jjic.loc[['제주','인천'],:]

# 제주 숙소 등급 분포 비교 
all_ht_star = ht.star.value_counts() # DB에 저장된 모든 숙박의 등급 분포
jj_ht_star = ht[ht.city == '제주'].star.value_counts() #DB에 저장된 모든 제주 숙박의 등급 분포
jj_confirmed_star = table[table.city == '제주'].star.value_counts()# 구매된 제주도 숙박의 등급 분포 
all_confirmed_star = table.star.value_counts() #구매된 숙박들의 숙소 등급 분포

In [17]:
#구매된 제주도 숙박 리스트 
table_jj = table[table.city == '제주'][['name_kr','star']].sort_values(by = 'star', ascending = False)
table_jj

,name_kr,star
5538,제주 부영 호텔&리조트,5.0
7132,메종 글래드 제주,5.0
1017,제주 부영 호텔&리조트,5.0
2292,메종 글래드 제주,5.0
2293,메종 글래드 제주,5.0
2294,메종 글래드 제주,5.0
3883,메종 글래드 제주,5.0
4544,제주 부영 호텔&리조트,5.0
7751,제주 부영 호텔&리조트,5.0
5350,메종 글래드 제주,5.0


In [19]:
#groupby로 분포 확인 
star_count_jj = table_jj.groupby(by='star').count()
star_count_jj


,name_kr
star,
0.0,1
3.0,3
4.0,9
5.0,13


In [20]:
#시각화
fig7 = go.Figure(data=[go.Pie(labels=star_count_jj.index, values=star_count_jj.name_kr.values, hole=.3)])
fig7.update_layout(
    title_text='제주 지역 판매 상품 숙소 등급 비율' # title of plot\
)
fig7.show()


In [25]:
#제주지역 DB등록 숙소 등급 비율
#데이터 불러오기 
DB_jj_star = ht[ht.city == '제주'].groupby(by='star').count()

In [26]:
#시각화 제주 지역 DB 등록 숙소 등급 분포 
fig8 = go.Figure(data=[go.Pie(labels=DB_jj_star.index, values=DB_jj_star.name_kr.values, hole=.3)])
fig8.update_layout(
    title_text='제주 지역 DB 등록 숙소 등급 비율' # title of plot\
)
fig8.show()

대구, 경북 지역의 문제 
- 경북 지역은 분석팀의 분석 결과를 미루어 보았을 때, 유저 이탈율이 가장 높은 지역이었다. 
- 따라서 이탈율을 개선하기 위해 경북 지역의 판매 현황과 특징을 확인해 보도록 하자.
- 경북 지역의 경우 기본적으로 두가지 특징을 가지고 있다.
    - 대부분의 구매가 경주 소재 숙박에서 이루어진다. 
    - 등급이 높은 호텔이 인기를 끈다.
- 한편 최근 경북지역은 황리단길?의 개발을 바탕으로 전통과 현대의 조화를 이루어 많은 관광객들을 모으고 있다.
- 그러나 DB에 활성화 되어 있는 럭셔리 경주숙소가 많지 않아, 사람들이 경주를 확인한 페이지에서 많이 이탈했을 수 있다. 
    - 따라서 경주의 숙소를 많이 보강하거나, 
    - 경주 근처의 숙소를 살펴볼때는 거리기반 모델을 바탕으로 최대한 경주에서 가까운 숙박을 추천할 수 있어야 한다. 

In [27]:
# 대구 경북지역 숙박 판매를 분석하기 위해 필요한 데이터 테이블 불러오기 
dggb_ht = ht[(ht.city == '경북')|(ht.city == '대구')]
dggb_star = dggb_ht.star.value_counts() # 대구 경북 지역의 숙박 등급 분포 
dggb_confirmed = table[(table.city == '경북')|(table.city == '대구')] #대구 경북에서 구매된 숙박들 
dggb_confirmed_list = dggb_confirmed[['name_kr','star','address']].sort_values(by=['star','name_kr'],ascending = False)
dggb_confirmed_list

,name_kr,star,address
6435,라한셀렉트 경주,5.0,경상북도 경주시 보문로 338
7600,라한셀렉트 경주,5.0,경상북도 경주시 보문로 338
11745,라한셀렉트 경주,5.0,경상북도 경주시 보문로 338
11796,라한셀렉트 경주,5.0,경상북도 경주시 보문로 338
11801,라한셀렉트 경주,5.0,경상북도 경주시 보문로 338
11802,라한셀렉트 경주,5.0,경상북도 경주시 보문로 338
11827,라한셀렉트 경주,5.0,경상북도 경주시 보문로 338
11891,라한셀렉트 경주,5.0,경상북도 경주시 보문로 338
11905,라한셀렉트 경주,5.0,경상북도 경주시 보문로 338
11934,라한셀렉트 경주,5.0,경상북도 경주시 보문로 338


In [29]:
#대구경북지역 거래된 숙소 등급 분포 확인
gj_star = dggb_confirmed_list.groupby(by = 'star').count()
fig9 = go.Figure(data=[go.Pie(labels=gj_star.index, values=gj_star.name_kr.values, hole=.3)])
fig9.update_layout(
    title_text='대구 경북 지역 판매된 숙소 등급 분포' # title of plot\
)
fig9.show()

In [15]:
#대구 경북 지역 도시를 구별 하기 위한 함수
def dggbcity(value):
    if '경주' in value:
        value = '경주'
    elif '대구' in value:
        value = '대구'
    else:
        value = '포항'
    return value

dggb_confirmed_list['대구 경북 도시'] = dggb_confirmed_list['address'].apply(dggbcity)
dggb_confirmed_list

,name_kr,star,address,대구 경북 도시
6435,라한셀렉트 경주,5.0,경상북도 경주시 보문로 338,경주
7600,라한셀렉트 경주,5.0,경상북도 경주시 보문로 338,경주
11745,라한셀렉트 경주,5.0,경상북도 경주시 보문로 338,경주
11796,라한셀렉트 경주,5.0,경상북도 경주시 보문로 338,경주
11801,라한셀렉트 경주,5.0,경상북도 경주시 보문로 338,경주
11802,라한셀렉트 경주,5.0,경상북도 경주시 보문로 338,경주
11827,라한셀렉트 경주,5.0,경상북도 경주시 보문로 338,경주
11891,라한셀렉트 경주,5.0,경상북도 경주시 보문로 338,경주
11905,라한셀렉트 경주,5.0,경상북도 경주시 보문로 338,경주
11934,라한셀렉트 경주,5.0,경상북도 경주시 보문로 338,경주


In [16]:
#대구 경북 숙박 판매 비율 
dggb_cell = dggb_confirmed_list.groupby(by = '대구 경북 도시').count()

#시각화
fig3 = go.Figure(data=[go.Pie(labels=dggb_cell.index, values=dggb_cell.star.values, hole=.3)])
fig3.update_layout(
    title_text='대구 경북 지역 숙소 판매 비중' # title of plot\
)
fig3.show()


In [17]:
# 경주 지역 숙박 리스트 
dggb_ht['대구 경북 숙소'] = dggb_ht['address'].apply(dggbcity)
gj_ht = dggb_ht[dggb_ht['대구 경북 숙소']== '경주'][['name_kr','star']].sort_values(by = 'star',ascending = False)
gj_star = gj_ht.groupby(by ='star').count()
gj_star

,name_kr
star,
0,32
3,6
4,5
5,1


In [18]:
#경주 숙박 등급 비율
fig4 = go.Figure(data=[go.Pie(labels=gj_star.index, values=gj_star.name_kr.values, hole=.3)])
fig4.update_layout(
    title_text='경주 지역 숙소 등급 비율' # title of plot\
)
fig4.show()


나머지 주요 지역 보완책 => 바다야 우리와 함께 놀아 
- 이외 6위 이내 판매 수량을 보이는 지역은 경기/부산/강원/인천이다.
- 본 보고서에서는 서울과의 인접성이 높아 서울에서 숙박을 즐기기 용이한 경기지역보다, 나머지 지역에 대한 전략을 제시하려고 한다. 
- 부산/강원/인천의 공통점은 바다와 인접해 있는 지역이라는 것이다.
    - 부산은 해운대와 광안리 해수욕장,
    - 강원도는 최근 서핑객들이 몰리고 있는 추세고, 
    - 인천 지역도 월미도가 있어 바다를 즐기기 좋은 입지이다. 

- 최근 바다 인접 지역에 풀빌라가 많이 지어지고 있고, 바다 근처에 리조트도 많은 편이다. 그러나 호텔 등급이 없는 0성으로 측정되어 있을 가능성이 높은데 올마이투어는 0성급 숙박이 많은 판매량을 보이지 못하고 있다. 
- 많이 구매되지 않고 있는 풀빌라나 리조트를 잘 추천할 수 있다면 기존에 이용 중인 유저들에 올마이투어만의 가치를 제안에 잠재된 수요를 자극할 수 있을 것으로 기대

In [33]:
#필요한 데이터 불러오기 
all_ht_star = pd.DataFrame(all_ht_star)
all_confirmed_star = pd.DataFrame(all_confirmed_star)
all_ht_star.rename(columns={'star':'DB_star'}, inplace= True)
all_confirmed_star.rename(columns={'star':'order_star'}, inplace= True)
ratio_star = all_ht_star.join(all_confirmed_star)

# Feature engineering 비율 값 추가 
ratio_star['DB_ratio'] = ratio_star['DB_star']/len(ht)
ratio_star['order_ratio'] = ratio_star['order_star']/len(table)
ratio_star

,DB_star,order_star,DB_ratio,order_ratio
0,907,258,0.430266,0.044537
3,806,234,0.382353,0.040394
4,300,2190,0.142315,0.378042
5,95,3103,0.045066,0.535646


In [31]:
#필요한 데이터 불러오기
all_ht_star = ht.star.value_counts() # DB에 저장된 모든 숙박의 등급 분포
all_confirmed_star = table.star.value_counts() #구매된 숙박들의 숙소 등급 분포

In [34]:
#시각화

trace1 = go.Bar(x=ratio_star.index, y=ratio_star['DB_ratio'], name='DB 등록 숙박 등급  비율 분석')
trace2 = go.Bar(x=ratio_star.index, y=ratio_star['order_ratio'], name='구매 완료 숙박 등급 비율 분석')

data = [trace1, trace2]
layout = go.Layout(title='숙소 등급 별 비율 ',
                   xaxis_title_text='비율', # xaxis label\
                   yaxis_title_text='숙소 등급' )
fig6 = go.Figure(data=data, layout=layout)
pyo.iplot(fig)


유저 군에 대한 추정 

In [34]:
#몇 박 묵고 가는지 시각화 하기 위함
cnt_term = table.groupby(by='terms').count()

#시각화
fig10 = go.Figure(data=[go.Pie(labels=cnt_term.index, values=cnt_term.name_kr.values, hole=.3)])
fig10.update_layout(
    title_text='전체 결제 완료 중 1박이 차지 하는 비중' # title of plot\
)
fig10.show()